In [1]:
import sys
import re
import os
from pathlib import Path
from collections import namedtuple

import numpy as np

### Taking data set from Seeval 2016 - task 5 subset 1 : http://alt.qcri.org/semeval2016/task5/

In [2]:
import os
import sys
import pandas as pd
import numpy as np
import pickle
from collections import Counter, defaultdict
import re

pd.set_option('display.max_colwidth' , -1)

### English - Restaurant domain training data

In [3]:
import xml.etree.ElementTree as ET
english = ET.parse('../data/ABSA16_Restaurants_Train_SB1_v2.xml')
root = english.getroot()

FileNotFoundError: [Errno 2] No such file or directory: '../data/ABSA16_Restaurants_Train_SB1_v2.xml'

In [4]:
labeled_reviews = []
for review in root.findall("Review"):
 

    if review.find("sentences"):
        
        for sentence in review.find("sentences").findall('sentence'): 
            if sentence.find('Opinions'):
                #print(sentence[0].text)
                entry= {} ;   aspects = [] ;  polarities = []
                for opinion in sentence.find('Opinions').findall('Opinion'):
                    aspects.append(opinion.get('category'))
                    polarities.append(opinion.get('polarity'))

        
    
                entry["text"] = sentence[0].text
                entry["aspects"] , entry['polarities']  =  aspects , polarities
                labeled_reviews.append(entry)
            
eng_multi_aspects = pd.DataFrame(labeled_reviews)
print("there are",len(labeled_reviews),"reviews in this training set")

NameError: name 'root' is not defined

In [5]:
eng_multi_aspects.head()

,aspects,polarities,text
0,[RESTAURANT#GENERAL],[negative],"Judging from previous posts this used to be a good place, but not any longer."
1,[SERVICE#GENERAL],[negative],"We, there were four of us, arrived at noon - the place was empty - and the staff acted like we were imposing on them and they were very rude."
2,[SERVICE#GENERAL],[negative],"They never brought us complimentary noodles, ignored repeated requests for sugar, and threw our dishes on the table."
3,"[FOOD#QUALITY, FOOD#STYLE_OPTIONS]","[negative, negative]",The food was lousy - too sweet or too salty and the portions tiny.
4,[SERVICE#GENERAL],[negative],"After all that, they complained to me about the small tip."


### Dutch  - Restaurant domain training data

In [6]:
english = ET.parse('../data/restaurants_dutch_training.xml')
root = english.getroot()

labeled_reviews = []
for review in root.findall("Review"):
 

    if review.find("sentences"):
        
        for sentence in review.find("sentences").findall('sentence'): 
            if sentence.find('Opinions'):
                #print(sentence[0].text)
                entry= {} ;   aspects = [] ;  polarities = []
                for opinion in sentence.find('Opinions').findall('Opinion'):
                    aspects.append(opinion.get('category'))
                    polarities.append(opinion.get('polarity'))

        
    
                entry["text"] = sentence[0].text
                entry["aspects"] , entry['polarities']  =  aspects , polarities
                labeled_reviews.append(entry)
            
du_multi_aspects = pd.DataFrame(labeled_reviews)
print("there are",len(labeled_reviews),"reviews in this training set")

there are 1317 reviews in this training set


In [7]:
du_multi_aspects.head()

,aspects,polarities,text
0,[SERVICE#GENERAL],[negative],Lange wachttijd.
1,[FOOD#STYLE_OPTIONS],[negative],"Zelfde dessert, 2 dagen na mekaar."
2,[FOOD#QUALITY],[positive],Ontbijtbuffet was tip top in orde.
3,[RESTAURANT#PRICES],[negative],Niet goedkoop!
4,[RESTAURANT#GENERAL],[positive],Maar eens in het kasteelrestaurant aangekomen werd het een feest.


#### We will extract embeddings for text column using https://github.com/facebookresearch/LASER . Saving text column separately for this.

In [8]:
eng_multi_aspects[['text']].to_csv('../data/tatoeba/v1/english_resturant.txt' , header = None , index = None , mode = 'a')
du_multi_aspects[['text']].to_csv('../data/tatoeba/v1/dutch_resturant.txt' , header = None , index = None , mode = 'a')

In [9]:
LASER_PATH = ".."
sys.path.append(LASER_PATH + '/source')
sys.path.append(LASER_PATH + '/source/lib')

DATA_PATH = Path("../data/tatoeba/v1/")
CACHE_PATH = Path("cache/")
CACHE_PATH.mkdir(exist_ok=True)
MODEL_PATH = Path("../models")

os.environ["LASER"] = LASER_PATH 

In [10]:
SPACE_NORMALIZER = re.compile("\s+")
Batch = namedtuple('Batch', 'srcs tokens lengths')

In [11]:
from indexing import IndexLoad, IndexTextOpen, IndexTextQuery, IndexSearchKNN, IndexCreate, IndexSearchMultiple
from embed import SentenceEncoder, EncodeLoad, EncodeFile
from text_processing import Token, BPEfastApply

#### Following steps from https://medium.com/the-artificial-impostor/multilingual-similarity-search-using-pretrained-bidirectional-lstm-encoder-e34fac5958b0 for tokenization , BPE Fast and Embedding extractions 

In [12]:
Token(
    str(DATA_PATH / "english_resturant.txt"),
    str(CACHE_PATH / "english_resturant.txt"),
    lang="eng",
    romanize=False,
    lower_case=True, gzip=False,
    verbose=True, over_write=False)

 - Tokenizer: english_resturant.txt exists already


In [13]:
Token(
    str(DATA_PATH / "dutch_resturant.txt"),
    str(CACHE_PATH / "dutch_resturant.txt"),
    lang="nld",
    romanize=False,
    lower_case=True, gzip=False,
    verbose=True, over_write=False)

 - Tokenizer: dutch_resturant.txt exists already


In [14]:
bpe_codes = str(MODEL_PATH / "93langs.fcodes")
BPEfastApply(
    str(CACHE_PATH / "english_resturant.txt"),
    str(CACHE_PATH / "english_resturant.txt.bpe"),
    bpe_codes,
    verbose=True, over_write=False)

 - fast BPE: english_resturant.txt.bpe exists already


In [15]:
BPEfastApply(
    str(CACHE_PATH / "dutch_resturant.txt"),
    str(CACHE_PATH / "dutch_resturant.txt.bpe"),
    bpe_codes,
    verbose=True, over_write=False)

 - fast BPE: dutch_resturant.txt.bpe exists already


## Extract Setence Embeddings

In [16]:
encoder = SentenceEncoder(
    str(MODEL_PATH / "bilstm.93langs.2018-12-26.pt"),
    max_sentences=None,
    max_tokens=10000,
    cpu=False)

In [17]:
EncodeFile(
    encoder,
    str(CACHE_PATH / "dutch_resturant.txt.bpe"),
    str(CACHE_PATH / "dutch_resturant.txt.enc"),
    verbose=True, over_write=False)

 - Encoder: dutch_resturant.txt.enc exists already


In [18]:
EncodeFile(
    encoder,
    str(CACHE_PATH / "english_resturant.txt.bpe"),
    str(CACHE_PATH / "english_resturant.txt.enc"),
    verbose=True, over_write=False)

 - Encoder: english_resturant.txt.enc exists already


In [19]:
data_en, index_en = IndexCreate(
    str(CACHE_PATH / "english_resturant.txt.enc"), 'FlatL2', verbose=True, save_index=False)
data_du, index_du = IndexCreate(
    str(CACHE_PATH / "dutch_resturant.txt.enc"), 'FlatL2', verbose=True, save_index=False)

 - embedding: cache/english_resturant.txt.enc 1708 examples of dim 1024
 - creating FAISS index
 - embedding: cache/dutch_resturant.txt.enc 1317 examples of dim 1024
 - creating FAISS index


#### We will search 3 nearest neighbour of 10  dutch sentences in all 1700 english sentences

In [20]:
_, matched_indices = index_en.search(data_du, 3)

In [21]:
du_list =  np.array(du_multi_aspects[['text']].head(10))
eng_list = np.array(eng_multi_aspects[['text']])
len(eng_list)

1708

In [22]:
top1_correct, top3_correct = 0, 0
for i, ztitle in enumerate(du_list):
    print(
        "Dutch:    ", ztitle, "\n",
        #"Correct:    ", translated_titles[i], "\n",
        "Predict(1): ", eng_list[matched_indices[i, 0]], "\n",
        "Predict(2): ", eng_list[matched_indices[i, 1]], "\n",
        "Predict(3): ", eng_list[matched_indices[i, 2]], "\n",
        sep=""
    )

Dutch:    ['Lange wachttijd.']
Predict(1): ['Great staff.']
Predict(2): ['Great survice']
Predict(3): ['Beautiful experience.']

Dutch:    ['Zelfde dessert, 2 dagen na mekaar.']
Predict(1): ['Dessert: pure disaster.']
Predict(2): ['The entree was bland and small, dessert was not inspired.']
Predict(3): ['We thought the dessert would be better, Wrong!']

Dutch:    ['Ontbijtbuffet was tip top in orde.']
Predict(1): ['fine dining restaurant quality.']
Predict(2): ['The buffet had a nice selection.']
Predict(3): ['Wine list selection is good and wine-by-the-glass was generously filled to the top.']

Dutch:    ['Niet goedkoop!']
Predict(1): ['Also very inexpensive.']
Predict(2): ['And amazingly cheap.']
Predict(3): ['Not worth it.']

Dutch:    ['Maar eens in het kasteelrestaurant aangekomen werd het een feest.']
Predict(1): ['After really enjoying ourselves at the bar we sat down at a table and had dinner.']
Predict(2): ['I attended a holiday dinner at the restaurant, and the food was major

##### Because dataset of semeval is not exact translation of each other , some of the above results are not good. 

### Creating multi label classification task using LASER sentence embedding. 
We can have 6 aspect categories , present for each review. We will train a simple 1 layer Neural Network model using 1024 dimensional sentence embedding as input and 6 categories as output.  
Train the model on 1700 English sentences and Validate on 1300 Dutch sentences . We are getting around 85 % accuracy and 57% f1 score(Macro) 

In [23]:
def extract_aspects(lst_aspects):
    res = []
    for x in lst_aspects:
        res.append(x.split('#')[0])
    return res

In [24]:
eng_multi_aspects['aspects2'] = eng_multi_aspects['aspects'].apply(lambda x: extract_aspects(x))
eng_multi_aspects['aspects2'] = eng_multi_aspects['aspects2'].apply(lambda x : list(set(x)))

Using only entity part out of entity#attribute tuple for simlicity

In [25]:
eng_multi_aspects.head(2)

,aspects,polarities,text,aspects2
0,[RESTAURANT#GENERAL],[negative],"Judging from previous posts this used to be a good place, but not any longer.",[RESTAURANT]
1,[SERVICE#GENERAL],[negative],"We, there were four of us, arrived at noon - the place was empty - and the staff acted like we were imposing on them and they were very rude.",[SERVICE]


In [26]:
eng_multi_aspects.aspects2.apply(pd.Series).merge(eng_multi_aspects , right_index = True , left_index = True)\
.drop(['aspects' , 'polarities' , 'aspects2'] ,axis = 1)\
.melt(id_vars = ['text']).drop(['variable'] , axis = 1).dropna().value.value_counts()


FOOD          757
RESTAURANT    564
SERVICE       419
AMBIENCE      226
DRINKS        79 
LOCATION      28 
Name: value, dtype: int64

In [27]:
du_multi_aspects['aspects2'] = du_multi_aspects['aspects'].apply(lambda x: extract_aspects(x))
du_multi_aspects['aspects2'] = du_multi_aspects['aspects2'].apply(lambda x : list(set(x)))

In [28]:
du_multi_aspects.head(2)

,aspects,polarities,text,aspects2
0,[SERVICE#GENERAL],[negative],Lange wachttijd.,[SERVICE]
1,[FOOD#STYLE_OPTIONS],[negative],"Zelfde dessert, 2 dagen na mekaar.",[FOOD]


In [29]:
du_multi_aspects.aspects2.apply(pd.Series).merge(du_multi_aspects , right_index = True , left_index = True)\
.drop(['aspects' , 'polarities' , 'aspects2'] ,axis = 1)\
.melt(id_vars = ['text']).drop(['variable'] , axis = 1).dropna().value.value_counts()

FOOD          547
SERVICE       420
RESTAURANT    353
AMBIENCE      178
DRINKS        92 
LOCATION      27 
Name: value, dtype: int64

In [30]:
eng_multi_aspects.drop(columns=['aspects' , 'polarities'] , inplace=True)
du_multi_aspects.drop(columns=['aspects' , 'polarities'] , inplace=True)

In [31]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb  = MultiLabelBinarizer()
y_eng = mlb.fit_transform(eng_multi_aspects.aspects2)
y_du  = mlb.fit_transform(du_multi_aspects.aspects2)

In [32]:
mlb.classes_

array(['AMBIENCE', 'DRINKS', 'FOOD', 'LOCATION', 'RESTAURANT', 'SERVICE'],
      dtype=object)

In [33]:
y_du[0:5]

array([[0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0]])

In [38]:
##One layer neural network with ReLU activation
n_in, n_h, n_out, batch_size = 1024,  500 , 6, 64

model = nn.Sequential(nn.Linear(n_in, n_h),
                     nn.ReLU(),
                     nn.Linear(n_h, n_out))

In [35]:
import torch 
import torch
import torch.nn as nn

x_train,y_train,x_valid,y_valid = map(torch.FloatTensor, (data_en,y_eng, data_du,y_du))
n,c = x_train.shape
y_train = y_train.type(torch.FloatTensor)
y_valid = y_valid.type(torch.FloatTensor)


In [36]:
print(y_train.shape , y_valid.shape)
print(x_train.shape , x_valid.shape)

torch.Size([1708, 6]) torch.Size([1317, 6])
torch.Size([1708, 1024]) torch.Size([1317, 1024])


In [39]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=batch_size , shuffle=True)

valid_ds = TensorDataset(x_valid, y_valid )
valid_dl = DataLoader(valid_ds , batch_size= batch_size )

In [40]:
class WrappedDataLoader():
    def __init__(self, dl, func):
        self.dl = dl
        self.func = func
        
    def __len__(self): return len(self.dl)
    
    def __iter__(self):
        batches = iter(self.dl)
        for b in batches: yield(self.func(*b))

In [41]:
dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
def preprocess(x,y): return x.to(dev),y.to(dev)

train_dl = WrappedDataLoader(train_dl, preprocess)
valid_dl = WrappedDataLoader(valid_dl, preprocess)


In [42]:
from torch import optim
optimizer = optim.Adam(model.parameters() , lr = 1e-2)
loss_func = nn.BCEWithLogitsLoss()

In [43]:
model = model.to(dev)
loss_func = loss_func.to(dev)

def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    acc = torch.sum(rounded_preds == y).float() 
    return acc

In [44]:
def train_model(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0    
    model.train()
    ct = 0
    for x, y in iterator:
        optimizer.zero_grad()
        #text, text_lengths =x
        predictions = model(x)
        loss = criterion(predictions, y)
        acc = binary_accuracy(predictions, y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        ct = ct + x.shape[0]   
    return epoch_loss / len(iterator), epoch_acc / (ct*6)

In [45]:
def validate_model(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0 
    ct = 0
    model.eval()
    with torch.no_grad():
        for x ,y  in iterator:

            predictions = model(x)#.squeeze(1)
            loss = criterion(predictions,y)
            acc = binary_accuracy(predictions, y)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            ct = ct + x.shape[0]
        
    return epoch_loss / len(iterator), epoch_acc / (ct*6)

In [46]:
epochs = 5

best_valid_loss = float('inf')
best_epoch = 0
        
for epoch in range(1, epochs + 1):
    train_loss , train_acc = train_model(model, train_dl, optimizer, loss_func)
    valid_loss , valid_acc = validate_model(model, valid_dl, loss_func)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        best_epoch = epoch
        #print(f'Best validation loss!! {best_valid_loss}')
        #torch.save(model.state_dict(), 'aspect_category.pt')

    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 02
	Train Loss: 0.408 | Train Acc: 81.40%
	 Val. Loss: 0.358 |  Val. Acc: 85.71%
Epoch: 03
	Train Loss: 0.264 | Train Acc: 89.51%
	 Val. Loss: 0.291 |  Val. Acc: 87.55%
Epoch: 04
	Train Loss: 0.199 | Train Acc: 92.21%
	 Val. Loss: 0.265 |  Val. Acc: 88.88%
Epoch: 05
	Train Loss: 0.157 | Train Acc: 94.02%
	 Val. Loss: 0.266 |  Val. Acc: 88.98%
Epoch: 06
	Train Loss: 0.131 | Train Acc: 94.76%
	 Val. Loss: 0.272 |  Val. Acc: 88.88%


In [47]:
test_preds = []
text_data = []
true_label = []

model.eval()
with torch.no_grad():
    for x ,y  in valid_dl:
        predictions = model(x)#.squeeze(1)
        rounded_preds = torch.round(torch.sigmoid(predictions))  #torch.round
        preds = rounded_preds.data.cpu().numpy()
        test_preds.append(preds)
        true_label.append(y.data.cpu().numpy())

In [48]:
aspects = mlb.classes_.tolist()

In [49]:
"""
Merging prediction value with original test data and observe the metrics on overall level
"""
dutch_pred = pd.DataFrame(np.vstack(test_preds) ,index=du_multi_aspects.index , columns= [a+'_pred' for a in aspects])
dutch_pred = pd.merge(du_multi_aspects, dutch_pred , left_index=True ,right_index = True)

from sklearn.metrics import f1_score , confusion_matrix , accuracy_score , precision_score , recall_score , roc_auc_score

In [50]:
print("F1 score",f1_score(y_du , dutch_pred[dutch_pred.columns[-6:]].as_matrix() , average='macro' ))
print("Accuracy score" , np.mean(y_du == dutch_pred[dutch_pred.columns[-6:]].as_matrix()))

F1 score 0.5785150146523276
Accuracy score 0.8887623386484435


/home/FRACTAL/swati.tiwari/anaconda3/envs/fastai38/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/home/FRACTAL/swati.tiwari/anaconda3/envs/fastai38/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/FRACTAL/swati.tiwari/anaconda3/envs/fastai38/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [52]:
dutch_pred.tail()

,text,aspects2,AMBIENCE_pred,DRINKS_pred,FOOD_pred,LOCATION_pred,RESTAURANT_pred,SERVICE_pred
1312,Niet goedkoop maar zeker zijn geld waard.,[RESTAURANT],0.0,0.0,1.0,0.0,1.0,0.0
1313,Zeer goede wijnen.,[DRINKS],0.0,1.0,0.0,0.0,0.0,0.0
1314,"wij zijn hier een paar keer per jaar, en steeds slaagt men erin te verrassen.",[RESTAURANT],0.0,0.0,0.0,0.0,1.0,0.0
1315,In de eerste plaats de creatieve gerechten en steeds terugkerend de top service.,"[FOOD, SERVICE]",0.0,0.0,0.0,0.0,0.0,1.0
1316,Een vaste waarde op topniveau,[RESTAURANT],0.0,0.0,0.0,0.0,1.0,0.0
